In [1]:
!pip install pulp folium pandas matplotlib

In [2]:
import pandas as pd
import folium
from pulp import *
import math


In [3]:
# Load the dataset
dataset_path = 'C:\\Users\\Samuel\\Documents\\ipynb_checkpoints\\customers_gwagwalada.csv'
data = pd.read_csv(dataset_path)

# Display the first few rows of the dataset
data.head()


,id,latitude,longitude,demand_bags,demand_packs
0,0,8.942050,7.081990,0,0
1,1,8.925081,7.052678,8,3
2,2,8.900276,7.067616,16,3
3,3,8.903489,7.050503,3,4
4,4,8.940240,7.094881,14,1


In [4]:
# Extract depot and customer data
depot = data[data['id'] == 0].iloc[0]
customers = data[data['id'] != 0]

# Extract coordinates and demands
depot_location = (depot['latitude'], depot['longitude'])
customer_locations = [(row['latitude'], row['longitude']) for index, row in customers.iterrows()]
customer_demands_bags = [row['demand_bags'] for index, row in customers.iterrows()]
customer_demands_packs = [row['demand_packs'] for index, row in customers.iterrows()]

# Calculate combined demand (assuming we treat 1 pack of bottled water as 1 unit for simplicity)
customer_demands = [bags + packs for bags, packs in zip(customer_demands_bags, customer_demands_packs)]

all_locations = [depot_location] + customer_locations
num_customers = len(customer_locations)
num_vehicles = 5  # or any appropriate number
vehicle_capacity = 220  # or any appropriate number


In [5]:
def calculate_distance(loc1, loc2):
    # havesine formula.
    R = 6371  # Earth radius in kilometers
    lat1, lon1 = math.radians(loc1[0]), math.radians(loc1[1])
    lat2, lon2 = math.radians(loc2[0]), math.radians(loc2[1])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c
distance_matrix = [[calculate_distance(loc1, loc2) for loc2 in all_locations] for loc1 in all_locations]


In [ ]:
# 3. ILP Model

prob = LpProblem("CVRP", LpMinimize)

# Sets
N = range(1, num_customers + 1)  # Customers
V = [0] + list(N)               # Vertices (depot + customers)
A = [(i, j) for i in V for j in V if i != j] # Arcs
K = range(num_vehicles)         # Vehicles

# Variables
x = LpVariable.dicts("x", (V, V, K), cat='Binary') # x[i][j][k] = 1 if vehicle k goes from i to j
u = LpVariable.dicts("u", N, lowBound=0, cat='Continuous') # u[i]: cumulative demand on the route for customer i

# Objective Function
prob += lpSum(distance_matrix[i][j] * x[i][j][k] for i in V for j in V for k in K if i != j), "Total_Distance"

# Constraints
for i in N:
    prob += lpSum(x[i][j][k] for j in V if j != i for k in K) == 1
for j in N:
    prob += lpSum(x[i][j][k] for i in V if i != j for k in K) == 1
for h in N:
    for k in K:
        prob += lpSum(x[i][h][k] for i in V if h != i) - lpSum(x[h][j][k] for j in V if h != j) == 0

# Subtour Elimination (MTZ)
for i in N:
    for j in N:
        if i != j:
            for k in K:
                prob += u[i] - u[j] + vehicle_capacity * x[i][j][k] <= vehicle_capacity - customer_demands_bags[j-1]

for k in K:
    prob += lpSum(customer_demands_bags[i-1] * x[i][j][k] for i in N for j in V if i != j) <= vehicle_capacity

# Solve the problem
prob.solve()
print("Status:", LpStatus[prob.status])
print("Optimal Total Distance =", value(prob.objective))

In [ ]:
# 4. Route Extraction & Visualization

# Extract Routes
routes = [[] for _ in range(num_vehicles)] 
for v in prob.variables():
    if v.varValue == 1 and v.name.startswith('x'):
        _, i, j, k = v.name.split('_')
        routes[int(k)].append(int(j))  # Append customer to route (we ignore return to depot)

# Add depot at the start and end of each route
for r in routes:
    r.insert(0, 0)
    r.append(0)

In [ ]:

# Visualize using Folium
map_center = depot_location
m = folium.Map(location=map_center, zoom_start=12)

# Add markers for depot and customers
folium.Marker(depot_location, popup="Depot", icon=folium.Icon(color='red', icon='industry', prefix='fa')).add_to(m)
for i, loc in enumerate(customer_locations):
    folium.Marker(loc, popup=f"Customer {i+1} (Demand {customer_demands_bags[i]} bags)").add_to(m)

# Draw the routes for each vehicle
colors = ['blue', 'green', 'purple', 'orange', 'red', 'darkred']  # Add more colors if needed
for k, route in enumerate(routes):
    route_locations = [all_locations[i] for i in route] 
    folium.PolyLine(route_locations, color=colors[k % len(colors)], weight=2.5, opacity=1, popup=f"Vehicle {k+1}").add_to(m)

# Show the map
m